In [38]:
#import the dataset 
from keras.datasets import fashion_mnist

In [39]:
#import the required libraries 
from matplotlib import pyplot
import numpy as np

In [40]:
#load the data into train and test 
(X_train,y_train),(X_test,y_test)=fashion_mnist.load_data()
#pyplot.imshow(X_train[2])

In [43]:
noOfImages = X_train.shape[0]
X_train = (1.0/255)*np.array([X_train[i].flatten() for i in range(0,noOfImages)])
X_train = np.array([X_train[i].flatten() for i in range(0,noOfImages)])
noOftestImages= X_test.shape[0]
X_test =(1.0/255)*np.array([X_test[i].flatten() for i in range(0,noOftestImages)])
X_test =np.array([X_test[i].flatten() for i in range(0,noOftestImages)])
print(X_test.shape)


(10000, 784)


In [44]:
print(X_train[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.00392157 0.         0.         0.05098039 0.28627451 

In [45]:
def softmax(X):
  X=np.exp(X)
  sum=np.sum(X,axis=0)
  return X/sum 


In [46]:
def sigmoidFunc(X):

  return 1.0/(1.0+np.exp(-X))
  #return res

In [47]:
def gDash(X):
  return sigmoidFunc(X)*(1-sigmoidFunc(X))

In [48]:
def forwardPropagation(X,parameters):
  preactivation={}
  activation={}
  activation['h0']=X.T
  #print(activation['h0'].shape)
  for k in range(1,noOfHiddenLayers+1):
    preactivation['a'+str(k)]=np.dot(parameters['W'+str(k)],activation['h'+str(k-1)])+parameters['b'+str(k)]
    activation['h'+str(k)]=sigmoidFunc(preactivation['a'+str(k)])
    #print('h size '+str(k),activation['h'+str(k)].shape)
    #print('a'+str(k),preactivation['a'+str(k)])
  preactivation['a'+str(noOfHiddenLayers+1)]=np.dot(parameters['W'+str(noOfHiddenLayers+1)],activation['h'+str(noOfHiddenLayers)])+parameters['b'+str(noOfHiddenLayers+1)]
  y=softmax(preactivation['a'+str(noOfHiddenLayers+1)])    
  #print("a last" ,preactivation['a'+str(noOfHiddenLayers+1)])
  return (preactivation,activation,y)



In [49]:
def backPropagation(parameters,activation,preactivation,yhat,X,y_train):
  grads={}
  eIndicator=np.zeros((10,X.shape[0]))
  eIndicator[y_train,np.arange(X.shape[0])]=1
  #print(y_train)
  #print(eIndicator)
  #print(eIndicator.shape)
  grads['a'+str(noOfHiddenLayers+1)]= -(eIndicator-yhat)
  #print(grads['a'+str(noOfHiddenLayers+1)].shape)
  for j in range(noOfHiddenLayers+1,0,-1):
    grads['W'+str(j)]= np.dot(grads['a'+str(j)],activation['h'+str(j-1)].T)
    #grads['W'+str(j)]= np.dot(activation['h'+str(j-1)],grads['a'+str(j)].T).T
    #print(grads['W'+str(j)].shape)
    grads['b'+str(j)]= np.sum(grads['a'+str(j)],axis=1,keepdims=True)
    ###grads['h'+str(j-1)]=np.dot(grads['W'+str(j)].T,grads['a'+str(j)]) #error
    grads['h'+str(j-1)]=np.dot(parameters['W'+str(j)].T,grads['a'+str(j)])
    #print('h'+str(j-1),grads['h'+str(j-1)].shape)
    #print('a'+str(j-1),preactivation['a'+str(j-1)].shape)
    if (j!=1):
      grads['a'+str(j-1)]=grads['h'+str(j-1)]*gDash(preactivation['a'+str(j-1)])
  return grads
#backPropagation()

In [50]:
def Loss(yhat,y_train,X):
  eIndicator=np.zeros((10,X.shape[0]))
  eIndicator[y_train,np.arange(X.shape[0])]=1
  eIndicator=eIndicator*yhat
  eIndicator=eIndicator.sum(axis=0)
  eIndicator=np.log(eIndicator)
  return -sum(eIndicator)


In [69]:

l=10 #output classes
noOfneuronsEach=[120,20]
noOfHiddenLayers=len(noOfneuronsEach)
inputNeuronSize=X_train.shape[1]
print(inputNeuronSize)
parameters={}
batchSize=600
iterations=100
eta=1
#eta=0.1
#input W
#initialization
parameters['W'+str(1)] =np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[0], inputNeuronSize))
parameters['b'+str(1)]= np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[0],1))
#parameters['W'+str(1)] =0+1.5*np.random.randn(noOfneuronsEach[0], inputNeuronSize)
#parameters['b'+str(1)]= 0+1.5*np.random.randn(noOfneuronsEach[0],1)
for i in range(2,noOfHiddenLayers+1):
  parameters['W'+str(i)] = np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[i-1], noOfneuronsEach[i-2]))
  parameters['b'+str(i)]= np.random.uniform(low=-0.5,high=0.5,size=(noOfneuronsEach[i-1],1))
  #parameters['W'+str(i)] = 0+1.5*np.random.randn(noOfneuronsEach[i-1], noOfneuronsEach[i-2])
  #parameters['b'+str(i)]= 0+1.5*np.random.randn(noOfneuronsEach[i-1],1)
#Output W
parameters['W'+str(noOfHiddenLayers+1)] = np.random.uniform(low=-0.5,high=0.5,size=(l, noOfneuronsEach[-1]))
parameters['b'+str(noOfHiddenLayers+1)]= np.random.uniform(low=-0.5,high=0.5,size=(l,1))
#parameters['W'+str(noOfHiddenLayers+1)] = 0+1.5*np.random.randn(l, noOfneuronsEach[-1])
#parameters['b'+str(noOfHiddenLayers+1)]= 0+1.5*np.random.randn(l,1)


784


In [70]:
def vanillagradDescent():
  t=0
  while(t < iterations):
    print("Epoch ",t)
    #print(parameters)
    mini=0
    while(mini<(noOfImages/batchSize)):
      #print("Batch ",mini)
      X_mini=X_train[(mini*batchSize):((mini+1)*batchSize-1)]
      y_mini=y_train[(mini*batchSize):((mini+1)*batchSize-1)]
      preactivation,activation,yhat=forwardPropagation(X_mini,parameters)
      gradients=backPropagation(parameters,activation,preactivation,yhat,X_mini,y_mini)
    #print("gradients",gradients)
      for i in range(1,noOfHiddenLayers+2):
        parameters['W'+str(i)] -=  eta*(1.0/X_mini.shape[0])*gradients['W'+str(i)]#parameters['W'+str(i)] -=  eta*(1.0/X_train.shape[0])*gradients['W'+str(i)]
        parameters['b'+str(i)] -= eta*(1.0/X_mini.shape[0])*gradients['b'+str(i)]#parameters['b'+str(i)] -= eta*(1.0/X_train.shape[0])*gradients['b'+str(i)]
      mini+=1
    _,_,yhat=forwardPropagation(X_train,parameters)
    print("loss after epoch ",Loss(yhat,y_train,X_train))
    
    #print(yhat)
    t+=1
vanillagradDescent()
#print(parameters)
#print(gradients)


Epoch  0
loss after epoch  47573.393426219605
Epoch  1
loss after epoch  41904.39835775272
Epoch  2
loss after epoch  33197.24226676189
Epoch  3
loss after epoch  30161.114459625456
Epoch  4


KeyboardInterrupt: ignored

In [68]:
def momentumGD():
  #initialization
  prevW={}
  prevb={}
  gamma=0.9
  prevW['W'+str(1)] =np.zeros((noOfneuronsEach[0], inputNeuronSize))
  prevb['b'+str(1)]= np.zeros((noOfneuronsEach[0],1))
  for i in range(2,noOfHiddenLayers+1):
    prevW['W'+str(i)] = np.zeros((noOfneuronsEach[i-1], noOfneuronsEach[i-2]))
    prevb['b'+str(i)]= np.zeros((noOfneuronsEach[i-1],1))
  prevW['W'+str(noOfHiddenLayers+1)] = np.zeros((l, noOfneuronsEach[-1]))
  prevb['b'+str(noOfHiddenLayers+1)]= np.zeros((l,1))
  t=0
  while(t<iterations):
    print("Epoch ",t)
    mini=0
    while(mini<(noOfImages/batchSize)):
      #print("Batch ",mini)
      X_mini=X_train[(mini*batchSize):((mini+1)*batchSize-1)]
      y_mini=y_train[(mini*batchSize):((mini+1)*batchSize-1)]
      preactivation,activation,yhat=forwardPropagation(X_mini,parameters)
      gradients=backPropagation(parameters,activation,preactivation,yhat,X_mini,y_mini)
    #print("gradients",gradients)
      for i in range(1,noOfHiddenLayers+2):
        w=gamma*prevW['W'+str(i)]+eta*(1.0/X_mini.shape[0])*gradients['W'+str(i)]
        b=gamma*prevb['b'+str(i)]+eta*(1.0/X_mini.shape[0])*gradients['b'+str(i)]
        parameters['W'+str(i)] -=  w
        parameters['b'+str(i)] -=  b
        prevW['W'+str(i)]=w
        prevb['b'+str(i)]=b
      mini+=1
    _,_,yhat=forwardPropagation(X_train,parameters)
    print("loss after epoch ",Loss(yhat,y_train,X_train))
    
    #print(yhat)
    t+=1
momentumGD()



Epoch  0
loss after epoch  28235.31184225386
Epoch  1
loss after epoch  23913.73563627984
Epoch  2
loss after epoch  22276.422719371847
Epoch  3
loss after epoch  20752.77516627167
Epoch  4
loss after epoch  19774.4372024492
Epoch  5
loss after epoch  18429.12712121991
Epoch  6
loss after epoch  17735.588868695533
Epoch  7
loss after epoch  17099.309072066786
Epoch  8
loss after epoch  17417.51882520029
Epoch  9
loss after epoch  16623.978858659517
Epoch  10
loss after epoch  16181.079187361598
Epoch  11
loss after epoch  17144.966061575873
Epoch  12
loss after epoch  16605.78465978326
Epoch  13
loss after epoch  15256.996168583055
Epoch  14
loss after epoch  14760.010781849554
Epoch  15
loss after epoch  14268.573224938957
Epoch  16


KeyboardInterrupt: ignored

In [64]:
def predict(X_test,y_test):
  _,_,y_hat=forwardPropagation(X_test,parameters)
  y_hat=y_hat.argmax(axis=0)
  correctPred=np.sum(y_hat==y_test)
  print("Accuracy is ", correctPred/X_test.shape[0]*100,"%")
  return (y_hat,correctPred)
print(predict(X_train,y_train))
print(predict(X_test,y_test))

Accuracy is  93.455 %
(array([9, 0, 0, ..., 3, 0, 5]), 56073)
Accuracy is  87.31 %
(array([9, 2, 1, ..., 8, 1, 5]), 8731)
